In [7]:
# import SQL module & create engine
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://LEO-LAPTOP/Springboard?driver=SQL+Server+Native+Client+11.0')

In [9]:
# get table names
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [17]:
# connect the engine
con = engine.connect()    

Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [24]:
import pandas as pd
rs1 = con.execute('select * from Facilities where membercost>0')
df1 = pd.DataFrame(rs1.fetchall())
df1.columns = rs1.keys()
df1

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


Q2: How many facilities do not charge a fee to members? 

In [25]:
rs2 = con.execute('select count("name") as "count" from Facilities where membercost=0')
df2 = pd.DataFrame(rs2.fetchall())
df2.columns = rs2.keys()
df2

,count
0,4


Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [27]:
rs3 = con.execute("""select facid, "name", membercost, monthlymaintenance from Facilities where membercost>0 
and membercost/monthlymaintenance<0.2""")
df3 = pd.DataFrame(rs3.fetchall())
df3.columns = rs3.keys()
df3

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.

In [30]:
rs4 = con.execute("""select * from Facilities where "name" like '%2'""")
df4 = pd.DataFrame(rs4.fetchall())
df4.columns = rs4.keys()
df4

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.

In [31]:
rs5 = con.execute("""select "name", monthlymaintenance
	   , case when monthlymaintenance>100 then 'expensive'
	     else 'cheap' end as costlevel
from Facilities""")
df5 = pd.DataFrame(rs5.fetchall())
df5.columns = rs5.keys()
df5

,name,monthlymaintenance,costlevel
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [32]:
rs6 = con.execute("""select firstname, surname from Members as a inner join 
	(select max(joindate) as latest
	from Members) as b
on a.joindate = b.latest""")
df6 = pd.DataFrame(rs6.fetchall())
df6.columns = rs6.keys()
df6

,firstname,surname
0,Darren,Smith


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [34]:
rs7 = con.execute("""select distinct "name" as facilityname, concat(m.firstname,' ',m.surname) as membername
from Members as m
inner join Bookings as b on m.memid=b.memid
inner join Facilities as f on b.facid=f.facid
where "name" like 'Tennis%'
group by "name", concat(m.firstname,' ',m.surname)
order by membername""")
df7 = pd.DataFrame(rs7.fetchall())
df7.columns = rs7.keys()
print(df7.head())

     facilityname    membername
0  Tennis Court 1    Anne Baker
1  Tennis Court 2    Anne Baker
2  Tennis Court 1  Burton Tracy
3  Tennis Court 2  Burton Tracy
4  Tennis Court 1  Charles Owen


Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [39]:
rs8 = con.execute("""select "name" as "facilityname", concat(firstname, ' ', surname) as personname
	   , case when b.memid=0 then guestcost*slots
			else membercost*slots end as cost, bookid
from Bookings as b
inner join Facilities as f on b.facid=f.facid
inner join Members as m on b.memid=m.memid
where starttime>='2012-09-14' and starttime<'2012-09-15' and 
(case when b.memid=0 then guestcost*slots
			else membercost*slots end)>30
order by cost desc""")
df8 = pd.DataFrame(rs8.fetchall())
df8.columns = rs8.keys()
print(df8)

      facilityname      personname   cost  bookid
0   Massage Room 2     GUEST GUEST  320.0    2946
1   Massage Room 1     GUEST GUEST  160.0    2937
2   Massage Room 1     GUEST GUEST  160.0    2940
3   Massage Room 1     GUEST GUEST  160.0    2942
4   Tennis Court 2     GUEST GUEST  150.0    2926
5   Tennis Court 1     GUEST GUEST   75.0    2920
6   Tennis Court 1     GUEST GUEST   75.0    2922
7   Tennis Court 2     GUEST GUEST   75.0    2925
8     Squash Court     GUEST GUEST   70.0    2948
9   Massage Room 1  Jemima Farrell   39.6    2941
10    Squash Court     GUEST GUEST   35.0    2949
11    Squash Court     GUEST GUEST   35.0    2951


Q9: This time, produce the same result as in Q8, but using a subquery.

In [40]:
rs9 = con.execute("""select "name" as "facilityname", concat(firstname, ' ', surname) as personname, cost, bookid  
from
(select memid, "name", guestcost*slots as "cost", bookid
from Bookings as b
inner join Facilities as f on b.facid=f.facid
where starttime>='2012-09-14' and starttime<'2012-09-15' and memid=0 and guestcost*slots>30
Union all
select memid, "name", membercost*slots as "cost", bookid
from Bookings as b
inner join Facilities as f on b.facid=f.facid
where starttime>='2012-09-14' and starttime<'2012-09-15' and memid<>0 and membercost*slots>30) as cost30
inner join Members on cost30.memid = Members.memid
order by cost desc""")
df9 = pd.DataFrame(rs9.fetchall())
df9.columns = rs9.keys()
print(df9)

      facilityname      personname   cost  bookid
0   Massage Room 2     GUEST GUEST  320.0    2946
1   Massage Room 1     GUEST GUEST  160.0    2937
2   Massage Room 1     GUEST GUEST  160.0    2940
3   Massage Room 1     GUEST GUEST  160.0    2942
4   Tennis Court 2     GUEST GUEST  150.0    2926
5   Tennis Court 1     GUEST GUEST   75.0    2920
6   Tennis Court 1     GUEST GUEST   75.0    2922
7   Tennis Court 2     GUEST GUEST   75.0    2925
8     Squash Court     GUEST GUEST   70.0    2948
9   Massage Room 1  Jemima Farrell   39.6    2941
10    Squash Court     GUEST GUEST   35.0    2949
11    Squash Court     GUEST GUEST   35.0    2951


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [41]:
rs10 = con.execute("""select "name",
	sum(case when memid=0 then guestcost*slots
	else membercost*slots end) as revenue
from Bookings as b
inner join Facilities as f on b.facid=f.facid
group by "name"
having sum(case when memid=0 then guestcost*slots
	else membercost*slots end)<1000""")
df10 = pd.DataFrame(rs10.fetchall())
df10.columns = rs10.keys()
print(df10)

            name  revenue
0     Pool Table    270.0
1  Snooker Table    240.0
2   Table Tennis    180.0


In [42]:
# Close connection
con.close()